In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
import pandas as pd
import pyodbc
import numpy as np
import os
import json

In [2]:
spark = SparkSession.builder.appName("PBPPlanBuilder").getOrCreate()

In [3]:
PLAN_YEAR = 2024
DR_TARGET_FOLDER = 'PBP_Benefits_2025_Results/'
BENEFIT_DATA_FILE = f'MedicareBenefits'
COMPARISON_RESULT_FILE = 'comparison_result'
MEDICARE_CRAWLED_DATA = 'MedicareCrawledData'
PBP_SOURCE_FOLDER = 'PBP_Benefits_2025/'
REGION_CODE = 'RegionCode'

In [9]:
def read_pd_from_csv_file(file_name):
    return pd.read_csv(DR_TARGET_FOLDER + file_name + '.csv')

def write_pd_to_csv(df, file_name):
    df.to_csv(DR_TARGET_FOLDER + file_name + '.csv', index=False)
    
def get_medicare_site_url(qid):
    contractid = qid[:5]
    planid = qid[5:8]
    segmentid = qid[8:]
    return f'https://www.medicare.gov/plan-compare/#/plan-details/{PLAN_YEAR}-{contractid}-{planid}-{int(segmentid)}?year={PLAN_YEAR}&lang=en#benefits'

def run_query_in_db(query, database):
    # Create a connection to the database
    with open('credentials.json', 'r') as file:
        data = json.load(file)
        connectionString = data[database]
        connectionString = (
            "DRIVER={ODBC Driver 17 for SQL Server};"
            "SERVER=docurobot-dev.cmba97i2bfdq.us-east-1.rds.amazonaws.com;"
            f"DATABASE={database};"
            "UID=alee;"
            "PWD=P@$$w0rd"
        )
        conn = pyodbc.connect(connectionString)
        return pd.read_sql_query(query, conn)

In [5]:
# load all required files
def load_csv(csv_file_path):
    return spark.read.format("csv") \
    .option("delimiter", "\t") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(csv_file_path)

# drop views from memory if the view exist
for t in spark.catalog.listTables():
    spark.catalog.dropTempView(t.name)

In [6]:
# drop views from memory if the view exist
for t in spark.catalog.listTables():
    spark.catalog.dropTempView(t.name)

# load PlanArea    
df_pbp_plan_area = load_csv(PBP_SOURCE_FOLDER + 'PlanArea.txt')
df_pbp_plan_area.createTempView('PlanArea')


# load PlanRegionArea    
df_pbp_plan_regions = load_csv(PBP_SOURCE_FOLDER + 'PlanRegionArea.txt')
df_pbp_plan_regions.createTempView('PlanRegionArea')

In [7]:
def convert_to_n_digit_str(num, digits):
    num_str = str(num)
    if len(num_str) < digits:
        num_str = ('0' * digits + num_str)[0-digits:]
    return num_str

In [10]:
countyfips_data_query = '''
select CountyFIPS, SSAFIPS, StateAbbr
from ZipcodeCounties where isactive = 1
group by CountyFIPS, SSAFIPS, StateAbbr
  '''

df_county_fips = run_query_in_db(countyfips_data_query, 'Docurobot')
df_county_fips['SSAFIPS'] = df_county_fips['SSAFIPS'].apply(lambda x: convert_to_n_digit_str(x, 5))
df_county_fips['CountyFIPS'] = df_county_fips['CountyFIPS'].apply(lambda x: convert_to_n_digit_str(x, 5))
write_pd_to_csv(df_county_fips, "CountyFIPS")


CMS_REGION_CODE_FILE = f'{DR_TARGET_FOLDER}{REGION_CODE}.csv'

if os.path.exists(CMS_REGION_CODE_FILE):
  df_region_code = pd.read_csv(CMS_REGION_CODE_FILE)
else:
  region_code_query = '''
  (select [COUNTY_CODE] as SSAFIPS
        ,[STATENAME]
        ,[COUNTY] as CountyName
        ,[MA_REGION_CODE] as ma_or_pdp_region_code
        ,[MA_REGION] as region
      ,'MA' as region_type
    FROM CMS.[dbo].[GeoLocation]
    group by [COUNTY_CODE]
        ,[STATENAME]
        ,[COUNTY]
        ,[MA_REGION_CODE]
        ,[MA_REGION]
		)
		union (
  select [COUNTY_CODE] as SSAFIPS
        ,[STATENAME]
        ,[COUNTY] as CountyName
        ,[PDP_REGION_CODE] as ma_or_pdp_region_code
        ,[PDP_REGION] as region, 
		'PD' as region_type
    FROM CMS.[dbo].[GeoLocation]
    group by [COUNTY_CODE]
        ,[STATENAME]
        ,[COUNTY]
        ,[PDP_REGION_CODE]
        ,[PDP_REGION]
		)
    '''
  df_region_code = run_query_in_db(region_code_query, 'CMS')
  df_region_code= pd.merge(df_region_code, df_county_fips, how='inner', on=[ 'SSAFIPS'])
  write_pd_to_csv(df_region_code, "RegionCode")


C:\Users\ajl0618\AppData\Local\Temp\ipykernel_8112\3848732615.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


In [11]:
df_pbp_plan_area = df_pbp_plan_area[['contract_year', 'pbp_a_hnumber', 'pbp_a_plan_identifier', 'segment_id2', 'county', 'stcd', 'county_code']].toPandas()
df_pbp_plan_regions = df_pbp_plan_regions[['contract_year', 'pbp_a_hnumber', 'pbp_a_plan_identifier', 'segment_id2', 'region_type','ma_or_pdp_region_code','region']].toPandas()

In [12]:
df_pbp_plan_regions.columns = ['PlanYear', 'ContractID', 'PlanID', 'SegmentID', 'region_type','ma_or_pdp_region_code','region']
df_pbp_plan_regions['PlanID'] = df_pbp_plan_regions['PlanID'].apply(lambda x: convert_to_n_digit_str(x, 3))
df_pbp_plan_regions['SegmentID'] = df_pbp_plan_regions['SegmentID'].apply(lambda x: convert_to_n_digit_str(x, 3))
df_pbp_plan_regions['ma_or_pdp_region_code'] = df_pbp_plan_regions['ma_or_pdp_region_code'].apply(lambda x: convert_to_n_digit_str(x, 2))
df_region_code['ma_or_pdp_region_code'] = df_region_code['ma_or_pdp_region_code'].apply(lambda x: convert_to_n_digit_str(x, 2))
df_pbp_plan_regions['QID'] = df_pbp_plan_regions.apply(lambda x: x['ContractID'] + x['PlanID'] + x['SegmentID'], axis=1)
df_pbp_plan_regions= pd.merge(df_pbp_plan_regions, df_region_code, how='inner', on=['region_type','ma_or_pdp_region_code','region'])
write_pd_to_csv(df_pbp_plan_regions, "PlanRegion")

In [13]:
# modify fields for plan area
df_pbp_plan_area.columns = ['PlanYear', 'ContractID', 'PlanID', 'SegmentID', 'CountyName','StateAbbr', 'SSAFIPS']
df_pbp_plan_area['PlanID'] = df_pbp_plan_area['PlanID'].apply(lambda x: convert_to_n_digit_str(x, 3))
df_pbp_plan_area['SegmentID'] = df_pbp_plan_area['SegmentID'].apply(lambda x: convert_to_n_digit_str(x, 3))
df_pbp_plan_area['SSAFIPS'] = df_pbp_plan_area['SSAFIPS'].apply(lambda x: convert_to_n_digit_str(x, 5))
df_pbp_plan_area['QID'] = df_pbp_plan_area.apply(lambda x: x['ContractID'] + x['PlanID'] + x['SegmentID'], axis=1)
df_pbp_plan_area= pd.merge(df_pbp_plan_area, df_county_fips, how='inner', on=['StateAbbr', 'SSAFIPS'])
write_pd_to_csv(df_pbp_plan_area, "PlanArea")

In [14]:
target_columns = ['PlanYear','ContractID','PlanID','SegmentID','QID','StateAbbr','CountyName','SSAFIPS','CountyFIPS']
df_plan_coverages = pd.concat([df_pbp_plan_regions[target_columns], df_pbp_plan_area[target_columns]])
write_pd_to_csv(df_plan_coverages, "PlanCoverage")

: 